<a href="https://colab.research.google.com/github/lituldust/Machine-Learning-Study/blob/main/Movie_Review_Datasets_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
train_data[0]

**Make Each Revies In The Same Length**

In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

**Create the Model**

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Train the Model**

In [9]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['acc'])

history = model.fit(train_data, train_labels, epochs = 10, validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 32s 49ms/step - loss: 0.4395 - acc: 0.7902 - val_loss: 0.3208 - val_acc: 0.8674
Epoch 2/10
625/625 [==============================] - 18s 29ms/step - loss: 0.2559 - acc: 0.8996 - val_loss: 0.3030 - val_acc: 0.8756
Epoch 3/10
625/625 [==============================] - 17s 27ms/step - loss: 0.1966 - acc: 0.9295 - val_loss: 0.3229 - val_acc: 0.8856
Epoch 4/10
625/625 [==============================] - 10s 17ms/step - loss: 0.1618 - acc: 0.9427 - val_loss: 0.3975 - val_acc: 0.8526
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1297 - acc: 0.9545 - val_loss: 0.2997 - val_acc: 0.8812
Epoch 6/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1088 - acc: 0.9643 - val_loss: 0.3449 - val_acc: 0.8776
Epoch 7/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0928 - acc: 0.9704 - val_loss: 0.3745 - val_acc: 0.8774
Epoch 8/10
625/625 [==============================] - 9s 14ms/st

In [10]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 6ms/step - loss: 0.5300 - acc: 0.8583
[0.5300263166427612, 0.8582800030708313]


**Making Predictions**

In [19]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN) [0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [20]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [24]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, 250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])
  return result[0]

positive_review = "That movie was so amazing! I really like and loved it, it'll great if i rewatch that awesome movie."
if predict(positive_review) >= 0.5:
  print("This is positive review")

negative_review = "That movie was sucked. I hated it and wouldn't watch it again, that was the worst movie i've ever watched."
if predict(negative_review) < 0.5:
  print("This is negative review")

1/1 [==============================] - 0s 22ms/step
[0.9128838]
This is positive review
1/1 [==============================] - 0s 21ms/step
[0.3819004]
This is negative review
